# Wave Glider Level 3 processing



## Produce Level 3a and Level 3b data

Level 3a data: (L3a_all)
- The only difference from Level 2 is that NaNs are removed by interpolation
- This is a "wave resolving' data set with modest QC (removal/interpolation of gaps<1 sec)

Level 3a subsets: (just interpolate gaps<1 sec and extract individual data sets)
- Make a velocity subset (L3a_vel)
- make a wave subset (L3a_wave)
- make a met subset (L3a_met)

Level 3b met data:
- Same as Level 3a but variables are averaged to 1 min
- 20 Hz variables are subsampled to 1 Hz time base
- Maybe in the future, will want to subsample all variables to a 1 min time base
- May not produce for all variables (eg, vechiles with met packages off)

Level 3b velocity data:
- extract velocity data and average to 5 minutes

Given plan above, I should reorganize code:
1. Do L3a processing in a L3a processor (interp small gaps, generate subsets, save files)
1. Do L3b processing in L3b processor (smooth, subsample, save subsets; primarily met and vel)



## Currently not sure what to do with wave data
- Maybe just save wave data as a separate data set, with no alteration

## Also, not sure how to handle data gaps when instruments were off
- For example, Kelvin has a max gap of 33 hours
- Thinking out loud, maybe I should check gap size, and, if significant, add NaNs back in after interpolating

This shows max gap:
```
gap_size = np.max(np.diff(ds.time_1Hz))
np.timedelta64(gap_size,'h')
```
Maybe I need to do something like
```
gaps = np.where(np.diff(ds.time_1Hz)>np.timedelta64(60,'s'))
```


In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib
import datetime as dt
################
# This allows us to import Tom_tools
import sys
sys.path.append('../../Tom_tools/') # you may need to adjust this path
# sys.path.append('../SWOT_IW_SSH/jtf/Tom_tools/') # you may need to adjust this path
import Tom_tools_v1 as tt
################

In [2]:
# %matplotlib inline
%matplotlib qt 
savefig = False # set to true to save plots as file
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 400
plt.close('all')
plt.rcParams['axes.xmargin'] = 0


__figdir__ = '../plots/WG_timeseries/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
# path = '/mnt/e/SMODE_data/pilot/WG/L2/'
path = '/mnt/c/D_drive/SMODE_data/pilot/WG/L2/'
WG = 'Kelvin'#'WHOI43'
file = 'SMODE_PFC_Wavegliders_'+WG+'.nc'

In [4]:
naninterp='True' # Interpolate NaNs

In [5]:
# This needs to be set before any dates are encoded as datetime64
# Because the time increment is so small, we get 'out of range' when starting from the standard datetime64 epoch of 1970
# matplotlib.dates.set_epoch('2000-01-01T00:00:00') 

In [6]:
%%time
ds = xr.open_dataset(path+file, engine = 'netcdf4', decode_times = True) #decode_times = False, 

CPU times: user 348 ms, sys: 258 ms, total: 606 ms
Wall time: 1.01 s


In [7]:
ds

<xarray.Dataset>
Dimensions:                     (time_1Hz: 450183, time_10Hz: 6840001, time_20Hz: 13680001, time_15min: 761, wave_frequency: 153, Workhorse_time: 450013, Workhorse_range: 50)
Coordinates: (12/13)
  * time_1Hz                    (time_1Hz) datetime64[ns] 2021-10-28T02:00:00...
    latitude_1Hz                (time_1Hz) float64 ...
    longitude_1Hz               (time_1Hz) float64 ...
  * time_10Hz                   (time_10Hz) datetime64[ns] 2021-10-28T02:00:0...
    altitude_10Hz               (time_10Hz) float32 ...
  * time_20Hz                   (time_20Hz) datetime64[ns] 2021-10-28T02:00:0...
    ...                          ...
  * wave_frequency              (wave_frequency) float32 0.04 0.045 ... 0.8
  * Workhorse_time              (Workhorse_time) datetime64[ns] 2021-10-28T02...
  * Workhorse_range             (Workhorse_range) float32 49.0 nan ... nan nan
    Workhorse_longitude         (Workhorse_time) float64 ...
    Workhorse_latitude          (Workhorse_time) float64 ...
    Workhorse_altitude          (Workhorse_time) float64 ...
Data variables: (12/51)
    INST_Gill                   float64 ...
    INST_upper_ctd              float64 ...
    INST_lower_ctd              float64 ...
    INST_WXT                    float64 ...
    INST_SITEX                  float64 ...
    INST_Nortek                 float64 ...
    ...                          ...
    Workhorse_heading           (Workhorse_time) float32 ...
    Workhorse_temperature       (Workhorse_time) float32 ...
    Workhorse_vel_east          (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_vel_north         (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_vel_up            (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_flag              (Workhorse_time, Workhorse_range) float32 ...
Attributes: (12/40)
    DOI:                           10.5067/SMODE-GLID3
    acknowledgement:               This work is a contribution to the S-MODE ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    summary:                       Wave Glider Kelvin was deployed by R/V Oce...
    uuid:                          74599383-706f-45cc-89be-bf85b6f531cc
    history:                       2022-06-09 Benjamin Greenwood - process in...
    date_created:                  2022-06-09T19:43:16Z
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    instruments:                   In Situ/Laboratory Instruments > Recorders...

In [8]:
# Raw met plot from WG:
fig, axs = plt.subplots(5, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(5,1,1)
h1, = plt.plot(ds.time_1Hz, ds.WXT_air_temperature)
h2, = plt.plot(ds.time_1Hz, ds.UCTD_sea_water_temperature)
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('Raw 1 Hz WXT measurements')

plt.subplot(5,1,2)
plt.plot(ds.time_1Hz, ds.WXT_relative_humidity)
plt.ylabel('[%]')
plt.legend(['Rel. Humidity'])

plt.subplot(5,1,3)
plt.plot(ds.time_15min, ds.wave_significant_height)
plt.ylabel('[m]')
plt.legend(['Sig. wave height'],loc='upper right')

plt.subplot(5,1,4)
plt.plot(ds.time_20Hz, ds.wind_speed)
plt.plot(ds.time_1Hz, ds.WXT_wind_speed)
plt.ylabel('[m/s]')
plt.legend(['Gill Wind speed','WXT wind speed'],loc='upper right')

plt.subplot(5,1,5)
plt.plot(ds.Workhorse_time, ds.Workhorse_altitude)
plt.ylabel('[m]')
plt.legend(['Workhorse altitude (from IMU/GPS)'],loc='upper right')

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met' + '.' +plotfiletype,**savefig_args)


In [9]:
# Raw met plot from WG:
fig, axs = plt.subplots(4, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(4,1,1)
plt.plot(ds.time_1Hz, ds.WXT_atmospheric_pressure)
plt.ylabel('[mbar]')
plt.legend(['Atm. pressure'])

plt.subplot(4,1,2)
try:
    plt.plot(ds.time_1Hz, ds.SGR4_longwave_flux)
    plt.ylabel('W/m^2')
    plt.legend(['Longwave radiation'])
except:
    plt.plot(ds.time_1Hz, ds.WXT_relative_humidity)
    plt.ylabel('%')
    plt.legend(['Relative humidity'])
    

plt.subplot(4,1,3)
try: 
    plt.plot(ds.time_1Hz, ds.SMP21_shortwave_flux)
    plt.ylabel('W/m^2')
    plt.legend(['Shortwave radiation'])
except:
    plt.plot(ds.time_15min, ds.wave_significant_height)
    plt.ylabel('m')
    plt.legend(['SWH'])

plt.subplot(4,1,4)
plt.plot(ds.time_1Hz, ds.WXT_rainfall_rate)
plt.ylabel('[mm/hr]')
plt.legend(['Precip. rate'])

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met2' + '.' +plotfiletype,**savefig_args)


In [10]:
# Raw met plot from WG:
fig, axs = plt.subplots(5, 1, sharex=True,figsize=(5,7))
fig.autofmt_xdate()
plt.subplot(5,1,1)
h1, = plt.plot(ds.time_1Hz, ds.WXT_air_temperature)
h2, = plt.plot(ds.time_1Hz, ds.UCTD_sea_water_temperature)
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('Wave Glider surface measurements')

try:
    plt.subplot(5,1,2)
    plt.plot(ds.time_1Hz, tt.run_avg1d(ds.SGR4_longwave_flux,15*60))
    plt.ylabel('W/m^2')
    plt.legend(['Longwave radiation'])
    plt.setp(plt.gca().get_xticklabels(), visible=False)

    plt.subplot(5,1,3)
    plt.plot(ds.time_1Hz, tt.run_avg1d(ds.SMP21_shortwave_flux,15*60))
    plt.ylabel('W/m^2')
    plt.legend(['Shortwave radiation'])
except:
    plt.subplot(5,1,3)


plt.subplot(5,1,4)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_wind_speed,15*60))
plt.ylabel('[m/s]')
plt.legend(['wind speed'],loc='upper right')

plt.subplot(5,1,5)
plt.plot(ds.time_15min, ds.wave_significant_height)
plt.ylabel('[m]')
plt.legend(['Sig. wave height'],loc='upper right')

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met3' + '.' +plotfiletype,**savefig_args)


In [11]:
# There is a limited number of NaNs in WXT wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz, np.isnan(ds.WXT_wind_speed.values))
plt.title('NaNs in WXT wind speed')

Text(0.5, 1.0, 'NaNs in WXT wind speed')

In [12]:
# Examine nans
ff = np.where(np.isnan(ds.WXT_wind_speed.values)==0)
t = ds.time_1Hz[ff]
print(str(np.round(100*(1-np.size(ff)/np.size(ds.time_1Hz)),3))+'% of values are NaN')

0.068% of values are NaN


In [13]:
gaps = np.where(np.diff(ds.time_1Hz)>np.timedelta64(60,'s'))

# This is the start of the 2nd gap
ds.time_1Hz[gaps[0][1]]

# I guess what I need to do is interpolate and then go back and set gaps to nans

<xarray.DataArray 'time_1Hz' ()>
array('2021-10-30T22:00:29.000000256', dtype='datetime64[ns]')
Coordinates:
    time_1Hz       datetime64[ns] 2021-10-30T22:00:29.000000256
    latitude_1Hz   float64 ...
    longitude_1Hz  float64 ...
Attributes:
    long_name:              Time recorded by Campbell CR6 datalogger at 1Hz
    standard_name:          time
    axis:                   T
    coverage_content_type:  coordinate

In [14]:
# There is also a limited number of NaNs in Gill wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_20Hz, np.isnan(ds.wind_speed.values))
plt.title('NaNs in Gill wind speed')

Text(0.5, 1.0, 'NaNs in Gill wind speed')

In [15]:
# How many nans?
ff = np.where(np.isnan(ds.wind_speed.values)==0)
t = ds.time_20Hz[ff]
print(str(np.round(100*(1-np.size(ff)/np.size(ds.time_20Hz)),3))+'% of values are NaN')

82.091% of values are NaN


In [16]:
gap_size = np.max(np.diff(ds.time_1Hz))
np.timedelta64(gap_size,'h')

numpy.timedelta64(33,'h')

## Make an averaged data set (say, 1 min)

### One thing to remember is that the time is not necessarily uniformly spaced.
But, for the WHOI WG, time_1Hz is ok:
```np.max(np.diff(ds.time_1Hz))``` is about 1 sec


In [17]:
# Make wind vector before smoothing
ds['WXT_wind_east'] = ds.WXT_wind_speed*np.cos(ds.WXT_wind_direction*np.pi/180)
ds['WXT_wind_north'] = ds.WXT_wind_speed*np.sin(ds.WXT_wind_direction*np.pi/180)
ds['wind_east']=ds.wind_speed*np.cos(ds.wind_direction*np.pi/180)
ds['wind_north']=ds.wind_speed*np.sin(ds.wind_direction*np.pi/180)


In [18]:
gapsize = np.timedelta64(60,'s')
gaps_1Hz = np.where(np.diff(ds.time_1Hz)>gapsize)
gaps_20Hz = np.where(np.diff(ds.time_20Hz)>gapsize)
gaps_WH = np.where(np.diff(ds.Workhorse_time)>gapsize)

print('1 Hz gaps exceeding '+str(gapsize) + ' start at:')
for n in range(np.size(gaps_1Hz[0][:])):
    print(ds.time_1Hz[gaps_1Hz[0][n]].values)


print('There are ' + str(np.size(gaps_1Hz[0][:])) + ' 1 Hz gaps exceeding '+str(gapsize))
print('There are ' + str(np.size(gaps_20Hz[0][:])) + ' 20 Hz gaps exceeding '+str(gapsize))
print('There are ' + str(np.size(gaps_WH[0][:])) + ' Workhorse gaps exceeding '+str(gapsize))

1 Hz gaps exceeding 60 seconds start at:
2021-10-30T15:04:43.000003968
2021-10-30T22:00:29.000000256
2021-10-31T23:39:08.000001536
2021-11-02T06:23:44.000000896
There are 4 1 Hz gaps exceeding 60 seconds
There are 0 20 Hz gaps exceeding 60 seconds
There are 4 Workhorse gaps exceeding 60 seconds


In [19]:
np.diff(ds.time_20Hz)

array([49999744, 49999744, 49999616, ..., 49999616, 49999744, 49999744],
      dtype='timedelta64[ns]')

In [20]:
def make_var_list(ds_in,time_coord):
    """
    Find all the variables with a given time coord
 
    Parameters
    ----------
    ds_in : xarray.dataset
    time_coord : str

    Returns
    -------
    result : list of str
        list of vars meeting criterion
    """
    
    var_list = []  
    not_used = []  
    for var in ds_in.data_vars:
        try:
            if ds_in.data_vars.get(var).dims[0]==time_coord:
                var_list.append(var)
                print(var)
        except:
            not_used.append(var)

    return var_list

In [21]:
def interp_nans(var_list, ds_in, gaps):

    for var in var_list:
        var_raw = ds_in.data_vars.get(var)
        if naninterp=='True': # Interpolate NaNs
            var_raw
            ff = np.flatnonzero(np.isnan(var_raw)==0)
            t = ds_in[ds_in.data_vars.get(var).coords.dims[0]]
            t_noninterp = t[ff]
            var_noninterp = var_raw[ff]
            var_value = np.interp(t, t_noninterp, var_noninterp)
        else: #just provide raw data as input to smoothing
            var_value = var_raw

        var_value[gaps[0][:]]= np.nan # insert 1 nan to make gaps evident in plots
        locals()[var] = var_raw.rename(var) #locals()['string'] makes a variable with the name string
        print(var+' created, '+str(np.round(100*(1-np.size(ff)/np.size(t)),3))+'% of values are NaN'+', max ff=' + str(np.size(ff)))
        locals()[var].values = var_value
        ds_new[var] = locals()[var]
    return ds_new

In [22]:
def subset(var_list, ds_in):

    for var in var_list:
        var_raw = ds_in.data_vars.get(var)
        var_value = var_raw
        ff = np.flatnonzero(np.isnan(var_raw)==0)
        locals()[var] = var_raw.rename(var) #locals()['string'] makes a variable with the name string
        print(var+' created, '+str(np.round(100*(1-np.size(ff)/np.size(var_raw)),3))+'% of values are NaN'+', max ff=' + str(np.size(ff)))
        locals()[var].values = var_value
        ds_new[var] = locals()[var]
    return ds_new

In [23]:
ds_new = xr.Dataset().assign_attrs(ds.attrs)  # make empty xr.Dataset but copy attributes from original file
ds_WH = xr.Dataset().assign_attrs(ds.attrs)  # make empty xr.Dataset but copy attributes from original file


In [24]:
var_list = make_var_list(ds, 'time_1Hz')

UCTD_depth
UCTD_sea_water_temperature
UCTD_conductivity
UCTD_salinity
UCTD_density
WXT_rainfall_rate
WXT_atmospheric_pressure
WXT_air_temperature
WXT_relative_humidity
WXT_wind_speed
WXT_wind_direction
WXT_wind_east
WXT_wind_north


In [25]:
ds_new = interp_nans(var_list, ds, gaps_1Hz)

UCTD_depth created, 0.045% of values are NaN, max ff=449981
UCTD_sea_water_temperature created, 0.0% of values are NaN, max ff=450182
UCTD_conductivity created, 0.0% of values are NaN, max ff=450182
UCTD_salinity created, 0.0% of values are NaN, max ff=450182
UCTD_density created, 0.045% of values are NaN, max ff=449981
WXT_rainfall_rate created, 0.01% of values are NaN, max ff=450137
WXT_atmospheric_pressure created, 0.01% of values are NaN, max ff=450140
WXT_air_temperature created, 0.009% of values are NaN, max ff=450141
WXT_relative_humidity created, 0.01% of values are NaN, max ff=450140
WXT_wind_speed created, 0.068% of values are NaN, max ff=449875
WXT_wind_direction created, 0.095% of values are NaN, max ff=449755
WXT_wind_east created, 0.095% of values are NaN, max ff=449755
WXT_wind_north created, 0.095% of values are NaN, max ff=449755


In [26]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds_new.time_1Hz,ds_new.WXT_wind_east)
plt.plot(ds_new.time_1Hz,ds_new.WXT_wind_north)
plt.plot(ds_new.time_1Hz,ds_new.WXT_wind_speed)
plt.title('Averaged WXT wind')
plt.ylabel('m/s')
#plt.ylim(-2,2)
plt.legend(['U','V','speed'])


In [27]:
# Now find 20 Hz variables (IMU and Gill sonic)
var_list = make_var_list(ds, 'time_20Hz')

magnetic_heading
true_heading
true_heading_corrected
pitch
roll
u_waveglider
v_waveglider
w_waveglider
latitude
longitude
altitude
Gill_temperature
wind_speed
wind_direction
wind_vertical
wind_east
wind_north


In [28]:
ds_new = interp_nans(var_list, ds, gaps_20Hz)

magnetic_heading created, 34.184% of values are NaN, max ff=9003651
true_heading created, 34.184% of values are NaN, max ff=9003651
true_heading_corrected created, 34.184% of values are NaN, max ff=9003651
pitch created, 34.184% of values are NaN, max ff=9003651
roll created, 34.184% of values are NaN, max ff=9003651
u_waveglider created, 34.184% of values are NaN, max ff=9003651
v_waveglider created, 34.184% of values are NaN, max ff=9003651
w_waveglider created, 34.184% of values are NaN, max ff=9003651
latitude created, 34.184% of values are NaN, max ff=9003633
longitude created, 34.184% of values are NaN, max ff=9003633
altitude created, 42.527% of values are NaN, max ff=7862333
Gill_temperature created, 82.142% of values are NaN, max ff=2442997
wind_speed created, 82.091% of values are NaN, max ff=2449916
wind_direction created, 82.091% of values are NaN, max ff=2449916
wind_vertical created, 82.091% of values are NaN, max ff=2449916
wind_east created, 82.091% of values are NaN, m

In [29]:
var_list = make_var_list(ds, 'Workhorse_time')

Workhorse_roll
Workhorse_pitch
Workhorse_heading
Workhorse_temperature
Workhorse_vel_east
Workhorse_vel_north
Workhorse_vel_up
Workhorse_flag


In [30]:
ds_new = subset(var_list, ds)

Workhorse_roll created, 0.0% of values are NaN, max ff=450013
Workhorse_pitch created, 0.0% of values are NaN, max ff=450013
Workhorse_heading created, 0.0% of values are NaN, max ff=450013
Workhorse_temperature created, 0.0% of values are NaN, max ff=450013
Workhorse_vel_east created, 11.353% of values are NaN, max ff=19946151
Workhorse_vel_north created, 11.353% of values are NaN, max ff=19946151
Workhorse_vel_up created, 11.353% of values are NaN, max ff=19946151
Workhorse_flag created, 0.0% of values are NaN, max ff=22500650


In [31]:
var_list = make_var_list(ds, 'time_15min')
ds_new = subset(var_list, ds)

wave_significant_height
wave_significant_height created, 34.691% of values are NaN, max ff=497


In [32]:
var_list = make_var_list(ds, 'wave_frequency')
ds_new = subset(var_list, ds)

wave_Szz
wave_direction
wave_Szz created, 34.691% of values are NaN, max ff=76041
wave_direction created, 34.691% of values are NaN, max ff=76041


In [33]:
ds_new

<xarray.Dataset>
Dimensions:                     (time_1Hz: 450183, time_20Hz: 13680001, Workhorse_time: 450013, Workhorse_range: 50, time_15min: 761, wave_frequency: 153)
Coordinates:
  * time_1Hz                    (time_1Hz) datetime64[ns] 2021-10-28T02:00:00...
    latitude_1Hz                (time_1Hz) float64 ...
    longitude_1Hz               (time_1Hz) float64 ...
  * time_20Hz                   (time_20Hz) datetime64[ns] 2021-10-28T02:00:0...
  * Workhorse_time              (Workhorse_time) datetime64[ns] 2021-10-28T02...
    Workhorse_longitude         (Workhorse_time) float64 ...
    Workhorse_latitude          (Workhorse_time) float64 ...
    Workhorse_altitude          (Workhorse_time) float64 -8.112 -8.169 ... nan
  * Workhorse_range             (Workhorse_range) float32 49.0 nan ... nan nan
  * time_15min                  (time_15min) datetime64[ns] 2021-10-28T02:00:...
  * wave_frequency              (wave_frequency) float32 0.04 0.045 ... 0.8
Data variables: (12/41)
    UCTD_depth                  (time_1Hz) float64 0.3375 0.3375 ... 0.2382
    UCTD_sea_water_temperature  (time_1Hz) float64 14.2 14.2 ... 14.81 14.81
    UCTD_conductivity           (time_1Hz) float64 4.051 4.051 ... 4.003 4.004
    UCTD_salinity               (time_1Hz) float64 33.5 33.5 ... 32.54 32.54
    UCTD_density                (time_1Hz) float64 1.025e+03 ... 1.024e+03
    WXT_rainfall_rate           (time_1Hz) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                          ...
    Workhorse_vel_north         (Workhorse_time, Workhorse_range) float32 -0....
    Workhorse_vel_up            (Workhorse_time, Workhorse_range) float32 -0....
    Workhorse_flag              (Workhorse_time, Workhorse_range) float32 0.0...
    wave_significant_height     (time_15min) float32 2.369 2.375 ... 4.605 nan
    wave_Szz                    (wave_frequency, time_15min) float32 0.2049 ....
    wave_direction              (wave_frequency, time_15min) float32 324.3 .....
Attributes: (12/40)
    DOI:                           10.5067/SMODE-GLID3
    acknowledgement:               This work is a contribution to the S-MODE ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    summary:                       Wave Glider Kelvin was deployed by R/V Oce...
    uuid:                          74599383-706f-45cc-89be-bf85b6f531cc
    history:                       2022-06-09 Benjamin Greenwood - process in...
    date_created:                  2022-06-09T19:43:16Z
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    instruments:                   In Situ/Laboratory Instruments > Recorders...

In [34]:
var_list = make_var_list(ds_new, 'wave_frequency')
ds_waves = subset(var_list, ds_new)
var_list = make_var_list(ds_new, 'time_15min')
ds_waves = subset(var_list, ds_new)

wave_Szz
wave_direction
wave_Szz created, 34.691% of values are NaN, max ff=76041
wave_direction created, 34.691% of values are NaN, max ff=76041
wave_significant_height
wave_significant_height created, 34.691% of values are NaN, max ff=497


In [35]:
var_list = make_var_list(ds_new, 'time_1Hz')
ds_1Hz = subset(var_list, ds_new)

UCTD_depth
UCTD_sea_water_temperature
UCTD_conductivity
UCTD_salinity
UCTD_density
WXT_rainfall_rate
WXT_atmospheric_pressure
WXT_air_temperature
WXT_relative_humidity
WXT_wind_speed
WXT_wind_direction
WXT_wind_east
WXT_wind_north
UCTD_depth created, 0.001% of values are NaN, max ff=450179
UCTD_sea_water_temperature created, 0.001% of values are NaN, max ff=450179
UCTD_conductivity created, 0.001% of values are NaN, max ff=450179
UCTD_salinity created, 0.001% of values are NaN, max ff=450179
UCTD_density created, 0.001% of values are NaN, max ff=450179
WXT_rainfall_rate created, 0.001% of values are NaN, max ff=450179
WXT_atmospheric_pressure created, 0.001% of values are NaN, max ff=450179
WXT_air_temperature created, 0.001% of values are NaN, max ff=450179
WXT_relative_humidity created, 0.001% of values are NaN, max ff=450179
WXT_wind_speed created, 0.001% of values are NaN, max ff=450179
WXT_wind_direction created, 0.001% of values are NaN, max ff=450179
WXT_wind_east created, 0.001

In [36]:
var_list = make_var_list(ds_new, 'time_20Hz')
ds_20Hz = subset(var_list, ds_new)

magnetic_heading
true_heading
true_heading_corrected
pitch
roll
u_waveglider
v_waveglider
w_waveglider
latitude
longitude
altitude
Gill_temperature
wind_speed
wind_direction
wind_vertical
wind_east
wind_north
magnetic_heading created, 0.0% of values are NaN, max ff=13680001
true_heading created, 0.0% of values are NaN, max ff=13680001
true_heading_corrected created, 0.0% of values are NaN, max ff=13680001
pitch created, 0.0% of values are NaN, max ff=13680001
roll created, 0.0% of values are NaN, max ff=13680001
u_waveglider created, 0.0% of values are NaN, max ff=13680001
v_waveglider created, 0.0% of values are NaN, max ff=13680001
w_waveglider created, 0.0% of values are NaN, max ff=13680001
latitude created, 0.0% of values are NaN, max ff=13680001
longitude created, 0.0% of values are NaN, max ff=13680001
altitude created, 0.0% of values are NaN, max ff=13680001
Gill_temperature created, 0.0% of values are NaN, max ff=13680001
wind_speed created, 0.0% of values are NaN, max ff=1368

In [37]:
var_list = make_var_list(ds_new, 'Workhorse_time')
ds_WH = subset(var_list, ds_new)

Workhorse_roll
Workhorse_pitch
Workhorse_heading
Workhorse_temperature
Workhorse_vel_east
Workhorse_vel_north
Workhorse_vel_up
Workhorse_flag
Workhorse_roll created, 0.0% of values are NaN, max ff=450013
Workhorse_pitch created, 0.0% of values are NaN, max ff=450013
Workhorse_heading created, 0.0% of values are NaN, max ff=450013
Workhorse_temperature created, 0.0% of values are NaN, max ff=450013
Workhorse_vel_east created, 11.353% of values are NaN, max ff=19946151
Workhorse_vel_north created, 11.353% of values are NaN, max ff=19946151
Workhorse_vel_up created, 11.353% of values are NaN, max ff=19946151
Workhorse_flag created, 0.0% of values are NaN, max ff=22500650


In [42]:
ds_20Hz

<xarray.Dataset>
Dimensions:                     (time_1Hz: 450183, time_20Hz: 13680001, Workhorse_time: 450013, Workhorse_range: 50, time_15min: 761, wave_frequency: 153)
Coordinates:
  * time_1Hz                    (time_1Hz) datetime64[ns] 2021-10-28T02:00:00...
    latitude_1Hz                (time_1Hz) float64 ...
    longitude_1Hz               (time_1Hz) float64 ...
  * time_20Hz                   (time_20Hz) datetime64[ns] 2021-10-28T02:00:0...
  * Workhorse_time              (Workhorse_time) datetime64[ns] 2021-10-28T02...
    Workhorse_longitude         (Workhorse_time) float64 ...
    Workhorse_latitude          (Workhorse_time) float64 ...
    Workhorse_altitude          (Workhorse_time) float64 -8.112 -8.169 ... nan
  * Workhorse_range             (Workhorse_range) float32 49.0 nan ... nan nan
  * time_15min                  (time_15min) datetime64[ns] 2021-10-28T02:00:...
  * wave_frequency              (wave_frequency) float32 0.04 0.045 ... 0.8
Data variables: (12/41)
    UCTD_depth                  (time_1Hz) float64 0.3375 0.3375 ... 0.2382
    UCTD_sea_water_temperature  (time_1Hz) float64 14.2 14.2 ... 14.81 14.81
    UCTD_conductivity           (time_1Hz) float64 4.051 4.051 ... 4.003 4.004
    UCTD_salinity               (time_1Hz) float64 33.5 33.5 ... 32.54 32.54
    UCTD_density                (time_1Hz) float64 1.025e+03 ... 1.024e+03
    WXT_rainfall_rate           (time_1Hz) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                          ...
    Workhorse_vel_north         (Workhorse_time, Workhorse_range) float32 -0....
    Workhorse_vel_up            (Workhorse_time, Workhorse_range) float32 -0....
    Workhorse_flag              (Workhorse_time, Workhorse_range) float32 0.0...
    wave_significant_height     (time_15min) float32 2.369 2.375 ... 4.605 nan
    wave_Szz                    (wave_frequency, time_15min) float32 0.2049 ....
    wave_direction              (wave_frequency, time_15min) float32 324.3 .....
Attributes: (12/40)
    DOI:                           10.5067/SMODE-GLID3
    acknowledgement:               This work is a contribution to the S-MODE ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    summary:                       Wave Glider Kelvin was deployed by R/V Oce...
    uuid:                          74599383-706f-45cc-89be-bf85b6f531cc
    history:                       2022-06-09 Benjamin Greenwood - process in...
    date_created:                  2022-06-09T19:43:16Z
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    instruments:                   In Situ/Laboratory Instruments > Recorders...

In [39]:
# Write averaged data to netcdf file
new_file = path+WG+'_L3a.nc'
print ('saving to ', new_file)
ds_new.to_netcdf(path=new_file)
ds_new.close()
print ('finished saving')

# Write averaged data to netcdf file
new_file = path+WG+'_L3a_1Hz_met.nc'
print ('saving to ', new_file)
ds_1Hz.to_netcdf(path=new_file)
ds_1Hz.close()
print ('finished saving')

# Write averaged data to netcdf file
new_file = path+WG+'_L3a_20Hz_met.nc'
print ('saving to ', new_file)
ds_20Hz.to_netcdf(path=new_file)
ds_20Hz.close()
print ('finished saving')

# Write averaged data to netcdf file
new_file = path+WG+'_L3a_ADCP.nc'
print ('saving to ', new_file)
ds_WH.to_netcdf(path=new_file)
ds_WH.close()
print ('finished saving')

saving to  /mnt/c/D_drive/SMODE_data/pilot/WG/L2/Kelvin_L3a.nc
finished saving
saving to  /mnt/c/D_drive/SMODE_data/pilot/WG/L2/Kelvin_L3a_1Hz_met.nc
finished saving
saving to  /mnt/c/D_drive/SMODE_data/pilot/WG/L2/Kelvin_L3a_20Hz_met.nc
finished saving
saving to  /mnt/c/D_drive/SMODE_data/pilot/WG/L2/Kelvin_L3a_ADCP.nc
finished saving


In [40]:
skfhdskahfka

NameError: name 'skfhdskahfka' is not defined

In [ ]:
# compare different time vectors to decide how to line up variables from different time bases
# ds.time_20Hz
# ds.time_1Hz
# ds.Workhorse_time

# Try the simple thing
# xy, x_ind, y_ind = np.intersect1d(ds.time_1Hz, ds.time_20Hz, assume_unique=True, return_indices=True)
# That doesn't work (no surprise) because the times don't line up exactly (possibly numerical precision issue)

# I think they are already nearly lined up
ds.time_1Hz[0:10:1]-ds.time_20Hz[0:200:20]
# so, try 0,20,40... in 20 Hz array

tfoo = ds.time_20Hz[0:-1:20]
#tfoo and time_1Hz are very close, within 10^-4 sec



In [ ]:
# ds.time_20Hz variables
# wdir is positive clockwise from North
# need to make U, V
Gill_U=ds.wind_speed*np.cos(ds.wind_direction*np.pi/180)
Gill_V=ds.wind_speed*np.sin(ds.wind_direction*np.pi/180)
Gill_wspd_low = tt.run_avg1d(ds.wind_speed, nsec*20)
Gill_U_low = tt.run_avg1d(Gill_U,nsec*20)
Gill_V_low = tt.run_avg1d(Gill_V,nsec*20)
pitch_low = tt.run_avg1d(ds.pitch,nsec*20)
roll_low = tt.run_avg1d(ds['roll'],nsec*20)

In [ ]:
# ds.time_1Hz variables
WXT_U=ds.WXT_wind_speed*np.cos(ds.WXT_wind_direction/np.pi/2)
WXT_V=ds.WXT_wind_speed*np.sin(ds.WXT_wind_direction/np.pi/2)
WXT_U_low = tt.run_avg1d(WXT_U,nsec)
WXT_V_low = tt.run_avg1d(WXT_V,nsec)
WXT_wspd_low = tt.run_avg1d(ds.WXT_wind_speed,nsec)
WXT_atmp_low = tt.run_avg1d(ds.WXT_air_temperature,nsec)
WXT_rh_low = tt.run_avg1d(ds.WXT_relative_humidity,nsec)
swr_low = tt.run_avg1d(ds.SMP21_shortwave_flux,nsec)
lwr_low = tt.run_avg1d(ds.SGR4_longwave_flux,nsec)
lat_low = tt.run_avg1d(ds.latitude_1Hz,nsec)
lon_low = tt.run_avg1d(ds.longitude_1Hz,nsec)

In [ ]:
# Workhorse_time variables
U_low=tt.run_avg2d(ds.Workhorse_vel_east,nsec,1)
V_low=tt.run_avg2d(ds.Workhorse_vel_north,nsec,1)
W_low=tt.run_avg2d(ds.Workhorse_vel_up,nsec,1)

In [ ]:
# Subsample smoothed 20 Hz vars to 1 Hz
Gill_wspd_low = Gill_wspd_low[0:-1:20]
Gill_U_low = Gill_U_low[0:-1:20]
Gill_V_low = Gill_V_low[0:-1:20]
pitch_low = pitch_low[0:-1:20]
roll_low = roll_low[0:-1:20]


In [ ]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz[0:-1:],pitch_low)
plt.plot(ds.time_1Hz[0:-1:],roll_low)
plt.title('1-hr average pitch and roll')
plt.ylabel('Degrees')
plt.ylim(-2,2)

if savefig:
    plt.savefig(__figdir__+WG+'_mean_pitch_roll' + '.' +plotfiletype,**savefig_args)


In [ ]:
plt.ylim(-2,2)

In [ ]:
new_file = path+WG+'_L2b_met.nc'
print ('saving to ', new_file)
ds_new.to_netcdf(path=new_file)
ds_new.close()
print ('finished saving')

In [ ]:
WG

In [ ]:
# this is broken for now
fig, axs = plt.subplots(1, 1)
M=151
tt.spectrum_band_avg(wspd_interp,1/20,M,winstr=None,plotflag=True,ebarflag=None)
tt.spectrum_band_avg(WXT_wspd_interp,1,M,winstr=None,plotflag=True,ebarflag=False)
plt.title('Gill, WXT wind speed')
plt.xlabel('Hz')
plt.ylabel('Spectral density [m$^2$/s$^2$/Hz]')
plt.legend(['Gill Wind speed','WXT wind speed'],loc='upper right')
if savefig:
    plt.savefig(__figdir__+WG+'_wsp_spectra' + '.' +plotfiletype,**savefig_args)


/home/jtomf/miniconda3/lib/python3.9/site-packages/ipykernel/eventloops.py:105: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


## Preparing for flux computation

Inputs for COARE 3.5:  
    u = ocean surface relative wind speed (m/s) at height zu(m)  
    t = bulk air temperature (degC) at height zt(m)  
    rh = relative humidity (%) at height zq(m)  
    ts = sea water temperature (degC) - see jcool below  
    P = surface air pressure (mb) (default = 1015)  
    Rs = downward shortwave radiation (W/m^2) (default = 150)  
    Rl = downward longwave radiation (W/m^2) (default = 370)  
    zu = wind sensor height (m) (default = 18m)  
    zt = bulk temperature sensor height (m) (default = 18m)  
    zq = RH sensor height (m) (default = 18m)  
    lat = latitude (default = 45 N)  
    zi = PBL height (m) (default = 600m)  
    rain = rain rate (mm/hr)  
    cp = phase speed of dominant waves (m/s)  
    sigH =  significant wave height (m)  
    jcool = cool skin option (default = 1 for bulk SST)  
    
Note: I don't see an input for SST depth--> That's because there is no Warm Layer correction in this version of the code.  

Here is a link to an older matlab version that does have it:  
https://github.com/carsonwitte/Falkor-DWL-Code  

It wouldn't be that hard to add the WL correction-- basically, estimate fluxes, estimate WL correction, and re-estimate fluxes.  


  
  
Required inputs:  
u	zu	t	zt	rh	zq	P	ts	Rs	Rl	lat	zi	rain	cp	sigH


This could be a good way to save the output, but maybe not:  

```
A = coare35vn(u, ta, rh, ts, P=Pa, Rs=rs, Rl=rl, zu=16, zt=16, zq=16,
                lat=Lat, zi=ZI, rain=Rain, jcool=1)
fnameA = os.path.join(path,'test_35_output_py_04022022.txt')
A_hdr = 'usr\ttau\thsb\thlb\thlwebb\ttsr\tqsr\tzot\tzoq\tCd\t'
A_hdr += 'Ch\tCe\tL\tzet\tdter\tdqer\ttkt\tRF\tCdn_10\tChn_10\tCen_10'
np.savetxt(fnameA,A,fmt='%.18e',delimiter='\t',header=A_hdr)
```
